Check Cluster State
====
This notebook check the state of ES cluster and sends mails to all the people substribed to that alert.
It is run once per hour from a cron job.

In [1]:
import subscribers, alerts
import requests

In [2]:
r=requests.get('http://cl-analytics.mwt2.org:9200/_cluster/health')
res=r.json()
print(res)

{'timed_out': False, 'status': 'red', 'number_of_pending_tasks': 22, 'delayed_unassigned_shards': 0, 'relocating_shards': 0, 'active_primary_shards': 7799, 'number_of_nodes': 19, 'task_max_waiting_in_queue_millis': 22810, 'cluster_name': 'atlas', 'number_of_data_nodes': 12, 'unassigned_shards': 3930, 'active_shards_percent_as_number': 77.29083665338645, 'number_of_in_flight_fetch': 21810, 'active_shards': 13580, 'initializing_shards': 60}


In [3]:
if res['status'] != 'green':
    S = subscribers.subscribers()

A = alerts.alerts()

if res['status'] == 'red':
    testName = 'Cluster is in RED'
    subscribersToRed =  S.getSubscribers(testName)
    for subscriber in subscribersToRed:
        body = 'Dear ' + subscriber[0]+',\n\n'
        body = body + '\tthis mail is to let you that the University of Chicago Elasticsearch cluster is in RED.\n'
        body = body + '\nBest regards,\nATLAS AAS'
        body = body + '\n\n To change your alerts preferences please you the following link:\n'+subscriber[2]
        print(subscriber)
        A.sendMail(testName, subscriber[1], body)
        A.addAlert(testName, subscriber[0],'simply red.')
if res['status'] == 'yellow':
    testName = 'Cluster is in YELLOW'
    subscribersToYellow = S.getSubscribers(testName)
    for subscriber in subscribersToYellow:
        body = 'Dear ' + subscriber[0]+',\n\n'
        body = body + '\tthis mail is to let you that the University of Chicago Elasticsearch cluster is in YELLOW.'
        body = body + 'There is '+str(res['unassigned_shards']) + ' unassigned shards on '+str(res['number_of_nodes']) + ' nodes.\n'
        body = body + '\nBest regards,\nATLAS AAS'
        body = body + '\n\n To change your alerts preferences please you the following link:\n'+subscriber[2]
        print(subscriber)
        A.sendMail(testName, subscriber[1], body)
        A.addAlert(testName, subscriber[1], str(res['unassigned_shards']) + ' unassigned shards on '+str(res['number_of_nodes']) + ' nodes.' )


['Ilija Vukotic', 'ilijav@gmail.com', 'https://docs.google.com/forms/d/e/1FAIpQLSfwwtAvMrqp4Ot_LYfmNu75_v33dtAxiXg7ZvVdn1X5v7TEgg/viewform?edit2=2_ABaOnuezuteui57-PrydNWrUuZf5fmChNqtjEeDab6h5V6lik_-x790uKsPu5Q']
Content-Type: text/plain; charset="us-ascii"
MIME-Version: 1.0
Content-Transfer-Encoding: 7bit
Subject: Cluster is in RED
From: AAAS@mwt2.org
To: ilijav@gmail.com

Dear Ilija Vukotic,

	this mail is to let you that the University of Chicago Elasticsearch cluster is in RED.

Best regards,
ATLAS AAS

 To change your alerts preferences please you the following link:
https://docs.google.com/forms/d/e/1FAIpQLSfwwtAvMrqp4Ot_LYfmNu75_v33dtAxiXg7ZvVdn1X5v7TEgg/viewform?edit2=2_ABaOnuezuteui57-PrydNWrUuZf5fmChNqtjEeDab6h5V6lik_-x790uKsPu5Q
['Lincoln Bryant', 'LINCOLNB@UCHICAGO.EDU', 'https://docs.google.com/forms/d/e/1FAIpQLSfwwtAvMrqp4Ot_LYfmNu75_v33dtAxiXg7ZvVdn1X5v7TEgg/viewform?edit2=2_ABaOnufEkC3ZkGEJM7YrjIK5M9zZCa8Fg0C58snIuKzECJRbScqS5po96bBVKA']
Content-Type: text/plain; charset